# Building a Gradio Frontend 

## Installing packages

In [1]:
!pip install keras_nlp kaggle gradio tensorflow

In [2]:
import keras_nlp
import os
import tensorflow as tf

2024-11-02 18:25:11.752855: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-02 18:25:12.091223: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-02 18:25:12.405681: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730571912.649290   37282 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730571912.717301   37282 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-02 18:25:13.338070: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

## Loading the model from kaggle

In [3]:
#For safety, add this to kaggle or google secrets
os.environ['KAGGLE_USERNAME'] = 'ayomidezulkazeem' #username
os.environ['KAGGLE_KEY'] = '245a313b018fe8355198d9c48413b991' #kaggle_key

In [4]:
import kagglehub

# Download latest version
path = kagglehub.model_download("ayomidezulkazeem/hydrosense/keras/gemma-hydrosense-instruct-2b")

print("Path to model files:", path)

Path to model files: /home/jupyter/.cache/kagglehub/models/ayomidezulkazeem/hydrosense/keras/gemma-hydrosense-instruct-2b/1


In [5]:
print("Model path:", path)

Model path: /home/jupyter/.cache/kagglehub/models/ayomidezulkazeem/hydrosense/keras/gemma-hydrosense-instruct-2b/1


In [6]:
model_path = "/home/jupyter/.cache/kagglehub/models/ayomidezulkazeem/hydrosense/keras/gemma-hydrosense-instruct-2b/1"
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(model_path)

2024-11-02 18:25:20.979637: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2024-11-02 18:25:21.020925: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2097152000 exceeds 10% of free system memory.
2024-11-02 18:25:24.845235: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2097152000 exceeds 10% of free system memory.
2024-11-02 18:25:25.250365: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2097152000 exceeds 10% of free system memory.
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
2024-11-02 18:25:44.663163: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2097152000 exceeds 10% of free system memory.


## Post-Processing the Output

In [7]:
def post_process_output(prompt, generated_text):
    # Remove the prompt if it's repeated at the beginning of the answer
    answer = generated_text.strip()
    if answer.startswith(prompt):
        answer = answer[len(prompt):].strip()

    # Remove any leading colons or whitespace
    answer = answer.lstrip(':')

    # Ensure the answer starts with a capital letter
    answer = answer.capitalize()

    # Ensure the answer ends with a period if it doesn't already
    if not answer.endswith('.'):
        answer += '.'

    return f"{answer}"

In [8]:
prompt = "When is the next maintenance due for Lift Station-485083 based on its last service date?"
generated_text = gemma_lm.generate(prompt, max_length=100)

# Apply post-processing
formatted_output = post_process_output(prompt, generated_text)
print(formatted_output)

I0000 00:00:1730571993.558518   37488 service.cc:148] XLA service 0x7ff0af000980 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730571993.561311   37488 service.cc:156]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1730571993.659753   37488 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Based on the last service date of 2023-02-14 and the bi-annual maintenance interval, the next service due for lift station-485083 is 2023-06-12.


## Gradio Interface

In [9]:
!pip install huggingface_hub

In [10]:
import gradio as gr
import time
from huggingface_hub import InferenceClient
import os

custom_css = """
@import url('https://fonts.googleapis.com/css2?family=Edu+AU+VIC+WA+NT+Dots:wght@400..700&family=Give+You+Glory&family=Sofia&family=Sunshiney&family=Vujahday+Script&display=swap');
.gradio-container, .gradio-container * {
     font-family: "Playfair Display", serif;
  font-optical-sizing: auto;
  font-weight: <weight>;
  font-style: normal;
}
"""
js = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') === 'light') {
        url.searchParams.set('__theme', 'light');
        window.location.href = url.href;
    }
}
"""


previous_sessions = []
memory = [{}]



def inference(prompt_text):
  prompt_text = prompt_text
  generated_text = gemma_lm.generate(prompt_text)

  #Apply post-processing
  formatted_output = post_process_output(prompt_text, generated_text)
  print(formatted_output)

  #adding a bit of delay
  time.sleep(1)
  result = formatted_output
  sessions = add_session(prompt_text)
  return result, sessions


def remember(prompt, result):
    global memory
    # Store the session as a dictionary
    session = {'prompt': prompt, 'result': result}
    memory.append(session)

    # Update previous_sessions for display
    session_display = [f"Q: {s['prompt']} \nA: {s['result']}" for s in memory]
    
    return "\n\n".join(session_display)  # Return formatted sessions as a string


def add_session(prompt_text):
    global previous_sessions
    session_name = ' '.join(prompt_text.split()[:5])
    
    if session_name and session_name not in previous_sessions:
        previous_sessions.append(session_name)
        
    return "\n".join(previous_sessions)  # Return only the session logs as a string


def clear_sessions():
    global previous_sessions
    previous_sessions.clear()
    return "\n".join(previous_sessions)

def clear_fields():
    return "", ""  # Return empty strings to clear the prompt and output fields


with gr.Blocks(theme='gradio/soft', css=custom_css) as demo:
    gr.Markdown("<center><h1>HydroFlow LLM Demo</h1></center>")
    gr.Markdown("<center><h3><i><em>Ask me anything on Wastewater or Stormwater!</em></i></h3></center>")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("## Previous Sessions")
            session_list = gr.Textbox(label="Sessions", value="\n".join(previous_sessions), interactive=False, lines=4, max_lines=20)
            add_button = gr.Button("New Session")
            clear_session = gr.Button("Clear Session")

        with gr.Column(scale=2):
            output = gr.Textbox(label="Result", lines=5, max_lines=20)
            prompt = gr.Textbox(label="Enter your Prompt here", max_lines=20)
            
            with gr.Row():
                generate_btn = gr.Button("Generate Answer", variant="primary", size="sm")
                reset_btn = gr.Button("Clear Content", variant="secondary", size="sm", elem_id="primary")


    generate_btn.click(
        fn=inference,
        inputs=[prompt],
        outputs=[output, session_list],
    )

    prompt.submit(
        fn=inference,
        inputs=[prompt],
        outputs=[output, session_list],
    )

    reset_btn.click(
        lambda: ("", ""),
        inputs=None,
        outputs=[prompt, output]
    )


    # Button to clear the prompt and output fields
    add_button.click(
        fn=clear_fields,  # Only call the clear_fields function
        inputs=None,      # No inputs needed
        outputs=[prompt, output]  # Clear the prompt and output fields
)


    clear_session.click(
        fn=clear_sessions,
        inputs=None,
        outputs=[session_list]
    )

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://65548fd86f47e3c37b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [11]:
!git config --global credential.helper store

In [12]:
from huggingface_hub import notebook_login

notebook_login()

In [13]:
!git clone https://huggingface.co/spaces/Zul001/gradio_llm_app

fatal: destination path 'gradio_llm_app' already exists and is not an empty directory.


In [7]:
cd gradio_llm_app

/home/jupyter/gradio_llm_app


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [13]:
!git config --global user.email "ayomidezulkazeem@gmail.com"
!git config --global user.name "Zulkazeem"

In [14]:
!git add app.ipynb 

In [15]:
!git commit -m "Initial commit"

[main 2a7b517] Initial commit
 1 file changed, 660 insertions(+)
 create mode 100644 app.ipynb


In [16]:
!git push

Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 4 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 7.04 KiB | 7.04 MiB/s, done.
Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
To https://huggingface.co/spaces/Zul001/gradio_llm_app
   2de767a..2a7b517  main -> main


In [ ]:
huggingface-cli repo create <Hydrosense>

Gradio_Interface.ipynb hd

In [ ]:
# Load your Keras model
model = tf.keras.models.load_model('gemma_lm.h5')